##### Copyright 2024 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 第4天 - 使用Gemini API进行Google Search数据落地

欢迎回到Kaggle 5天生成式AI课程！

在本教程中,你将使用[Google Search](https://google.com/)结果和Gemini API来进行一项称为grounding的技术,该技术将模型连接到可验证的信息源。使用search grounding类似于你本周早些时候实现的RAG系统,但Gemini API为你自动化了很多工作。模型会自动生成Google Search查询并执行搜索,从Google的网页索引中检索相关数据,并提供支持查询的搜索建议链接,以便用户可以验证信息来源。

## 启用计费

**重要提示!**

Google Search的grounding功能仅通过**"按使用付费"**账户的API提供。不过,你可以在[Google AI Studio](https://aistudio.google.com/)中**免费**尝试该功能,了解它的工作原理。在本指南中寻找 <img src="https://ai.google.dev/site-assets/images/marketing/home/icon-ais.png" style="height: 24px" height=24/> 链接可以直接前往该平台。

* 了解如何[启用计费](https://ai.google.dev/gemini-api/docs/billing#enable-cloud-billing)  
* 了解Google Cloud为新客户提供的[$300信用额度](https://cloud.google.com/free/docs/free-cloud-features)和[其他免费选项](https://cloud.google.com/free)
* 查看[价格页面](https://ai.google.dev/pricing)
* 或者在[Google AI Studio](https://aistudio.google.com/)中试用grounding功能,**无需**启用计费

如果你已经为API密钥启用了计费,请继续学习本指南,或者在AI Studio中打开示例提示来试用。

## Get set up

Start by installing and importing the Gemini API Python SDK.

In [ ]:
%pip install -q -U 'google-generativeai>=0.8.3'

You do not need to restart the kernel.

In [ ]:
import google.generativeai as genai
from IPython.display import Markdown, HTML, display

### Set up your API key

The `GOOGLE_API_KEY` environment variable can be set to automatically configure the underlying API. This works for both the official Gemini Python SDK and for LangChain/LangGraph. 

To run the following cell, your API key must be stored it in a [Kaggle secret](https://www.kaggle.com/discussions/product-feedback/114053) named `GOOGLE_API_KEY`.

If you don't already have an API key, you can grab one from [AI Studio](https://aistudio.google.com/app/apikey). You can find [detailed instructions in the docs](https://ai.google.dev/gemini-api/docs/api-key).

To make the key available through Kaggle secrets, choose `Secrets` from the `Add-ons` menu and follow the instructions to add your key or enable it for this notebook.

In [ ]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

If you received an error response along the lines of `No user secrets exist for kernel id ...`, then you need to add your API key via `Add-ons`, `Secrets` **and** enable it.

![Screenshot of the checkbox to enable GOOGLE_API_KEY secret](https://storage.googleapis.com/kaggle-media/Images/5gdai_sc_3.png)

### Explore available models

Search grounding is a tool available in the `-002` series of models. Find a model that supports it through the [`models.list`](https://ai.google.dev/api/models#method:-models.list) endpoint. You can also find more information about different model capabilities on [the models page](https://ai.google.dev/gemini-api/docs/models/gemini).

In [ ]:
for model in genai.list_models():
    if "002" in model.name:
        print(model.name)

## 使用search grounding

要启用search grounding,你需要将其指定为一个工具: `google_search_retrieval`。与其他工具一样,这可以作为参数提供给模型(用于所有聊天轮次或`generate_content`调用),也可以每次提供给`chat.send_message`。

<table align=left>
  <td>
    <a target="_blank" href="https://aistudio.google.com/app/prompts/1GTkO-gH4vd6G7LpBJ6Ay7U1OaJer7yDD"><img src="https://ai.google.dev/site-assets/images/marketing/home/icon-ais.png" style="height: 24px" height=24/> 在AI Studio中打开</a>
  </td>
</table>

In [ ]:
# Ask for information without search grounding.
model = genai.GenerativeModel("gemini-1.5-flash-002")

response = model.generate_content("When and where is Taylor Swift's next concert?")

Markdown(response.text)

Now try with grounding enabled.

<table align=left>
  <td>
    <a target="_blank" href="https://aistudio.google.com/prompts/14lDR0VjSni6BEUCZUBqj5PzTn3J194Th"><img src="https://ai.google.dev/site-assets/images/marketing/home/icon-ais.png" style="height: 24px" height=24/> Open in AI Studio</a>
  </td>
</table>

In [ ]:
# 创建一个启用了搜索数据落地功能的 Gemini 模型实例
model = genai.GenerativeModel(
    # 使用 gemini-1.5-flash-002 模型
    "gemini-1.5-flash-002",
    # 启用 Google 搜索检索工具
    tools="google_search_retrieval")

# 生成关于 Taylor Swift 演唱会的回答
# 模型会自动搜索并使用最新的网络信息
response = model.generate_content("When and where is Taylor Swift's next concert?")
# 获取第一个候选回答
rc = response.candidates[0]

# 将回答转换为 Markdown 格式并显示
# content.parts[0].text 获取生成的文本内容
Markdown(rc.content.parts[0].text)

If you receive a `429 Resource has been exhausted` error, you are likely using a free tier API key. You can choose to enable billing (but this will incur charges), or you can try the queries in Google AI Studio by following the links above.

### 响应元数据

当使用search grounding时,模型会返回额外的元数据,包括搜索建议链接、支持文档以及有关如何使用支持文档的信息。

每个"grounding chunk"代表从Google Search检索到的用于grounded生成请求的信息。点击URI链接将带你前往信息源。

In [ ]:
chunks = rc.grounding_metadata.grounding_chunks
for chunk in chunks:
    print(chunk)

作为响应的一部分,有一个独立的样式化HTML内容块,你可以用它链接回与生成内容相关的搜索建议。

In [ ]:
HTML(rc.grounding_metadata.search_entry_point.rendered_content)

元数据中的`grounding_supports`提供了一种方法,让你可以将使用的grounding chunks与生成的输出文本相关联。

In [ ]:
# 获取响应中的数据落地支持信息
# grounding_metadata.grounding_supports 包含了回答中使用的信息来源
supports = rc.grounding_metadata.grounding_supports

# 遍历并打印每个支持信息
# 每个 support 包含:
# - segment: 回答中的文本片段
# - grounding_chunk_indices: 对应的搜索结果索引
for support in supports:
    print(support)

These supports can be used to highlight text in the response, or build tables of footnotes.

In [ ]:
import io

# 创建一个字符串缓冲区用于构建 Markdown 文本
markdown_buffer = io.StringIO()

# 打印带有脚注标记的文本部分
markdown_buffer.write("Supported text:\n\n")
for support in supports:
    # 为每个支持的文本片段添加列表标记
    markdown_buffer.write(" * ")
    # 从原始响应文本中提取对应的文本片段
    markdown_buffer.write(
        response.text[support.segment.start_index : support.segment.end_index]
    )

    # 为每个文本片段添加对应的脚注引用标记
    for i in support.grounding_chunk_indices:
        chunk = chunks[i].web
        # 添加上标形式的引用编号
        markdown_buffer.write(f"<sup>[{i+1}]</sup>")

    # 添加换行
    markdown_buffer.write("\n\n")


# 打印引用来源
markdown_buffer.write("Citations:\n\n")
# 遍历所有搜索结果块,生成引用列表
# enumerate(chunks, start=1) 从1开始编号
for i, chunk in enumerate(chunks, start=1):
    # 创建带有链接的引用条目
    markdown_buffer.write(f"* {i}: [{chunk.web.title}]({chunk.web.uri})\n")


# 将构建好的 Markdown 文本转换为 Markdown 格式显示
Markdown(markdown_buffer.getvalue())

### 动态grounding

在可能无法提前知道是否启用search grounding的情况下,你可以为模型提供一个阈值,超过该阈值时它将使用search grounding。这在对话场景中很有帮助,因为并非每一轮对话都需要搜索数据来支持响应。

如果你知道是否需要在任何给定的聊天轮次中启用Search,你可以明确提供该工具。

<table align=left>
  <td>
    <a target="_blank" href="https://aistudio.google.com/prompts/1VBx_R16kNWa8g7lpLxQPx_08sFtd7tcd"><img src="https://ai.google.dev/site-assets/images/marketing/home/icon-ais.png" style="height: 24px" height=24/> 在AI Studio中打开</a>
  </td>
</table>

In [ ]:
# 创建一个不带搜索功能的基础 Gemini 模型实例
nosearch_model = genai.GenerativeModel("gemini-1.5-flash-002")
# 启动聊天会话
chat = nosearch_model.start_chat()

# 发送第一条消息 - 不使用搜索功能
# 这是一个简单的问候,不需要实时信息
r = chat.send_message("Hello friendly chatbot!")

# 发送第二条消息 - 仅在这一轮启用搜索功能
# 这个问题需要最新的体育赛事信息,所以启用搜索
r = chat.send_message(
    "Who took home the 2023 cricket world cup?", 
    tools="google_search_retrieval"  # 临时启用搜索工具
)

# 将回答转换为 Markdown 格式并显示
display(Markdown(r.text))

In [ ]:
HTML(r.candidates[0].grounding_metadata.search_entry_point.rendered_content)

或者你可以让Gemini API计算响应需要搜索支持的可能性,并定义使用的阈值。

In [ ]:
# 配置动态搜索设置
# dynamic_retrieval_score 是一个概率值,范围在 [0, 1] 之间
search_config = {
    "dynamic_retrieval_config": {
        "mode": "MODE_DYNAMIC",      # 设置动态模式
        "dynamic_threshold": 0.5     # 设置阈值为 0.5
    }
}

# 创建一个带有动态搜索配置的模型实例
maybe_search_model = genai.GenerativeModel(
    "gemini-1.5-flash-002", 
    tools={"google_search_retrieval": search_config}
)

# 启动聊天会话
chat = maybe_search_model.start_chat()

# 发送第一条消息(简单问候)
r = chat.send_message("Hello friendly chatbot!")
rc = r.candidates[0]
# 获取第一轮对话的搜索概率分数
score = rc.grounding_metadata.retrieval_metadata.google_search_dynamic_retrieval_score
print(f"First turn: {score=}")

# 发送第二条消息(需要实时信息的查询)
r = chat.send_message("Who took home the 2023 cricket world cup?")
rc = r.candidates[0]
# 获取第二轮对话的搜索概率分数
score = rc.grounding_metadata.retrieval_metadata.google_search_dynamic_retrieval_score
print(f"Second turn: {score=}")
print()

# 显示回答内容
display(Markdown(r.text))

In [ ]:
HTML(rc.grounding_metadata.search_entry_point.rendered_content)

## 延伸阅读

在使用search grounding时,你必须遵循一些特定要求,包括何时以及如何显示搜索建议,以及如何使用grounding链接。请务必阅读并遵循[search grounding能力指南](https://ai.google.dev/gemini-api/docs/grounding)和[搜索建议指南](https://ai.google.dev/gemini-api/docs/grounding/search-suggestions)中的详细内容。